In [24]:
import os
import time
import tensorflow.compat.v1 as tf
import pickle

tf.enable_eager_execution() # No need for session to be created. Function instances are run immediately. 

from waymo_open_dataset import dataset_pb2 as open_dataset
from google.cloud import storage

import concurrent.futures as concurr

# CONFIG
project = "Waymo3DObjectDetection"
bucket_name = 'waymo_open_dataset_v_1_2_0_individual_files'
suffix = '.tfrecord'
data_destination = os.getcwd() + "/data/"
download_batch_size = 1

def download_blob(blob, c):
    """
    blob = single file name
    c = file counter
    """
    fname = f"{data_destination}blob_{c}{suffix}"
    blob.download_to_filename(fname)
    return fname

def strip_frame(frame, idx, blob_idx):
    """Strip frame from garbage such as LIDAR data"""
    
    cam_dict = {}
    for i, camera in enumerate(["FRONT", "FRONT_LEFT", "SIDE_LEFT", "FRONT_RIGHT", "SIDE_RIGHT"]):
        cam_dict[camera] = {}
        cam_dict[camera]['image'] = frame.images[i].image
        cam_dict[camera]['velocity'] = frame.images[i].velocity
        cam_dict[camera]['labels'] = frame.camera_labels[i]
        
        cam_dict[camera]['context']={'stats':frame.context.stats, 
                           'name': frame.context.name, 
                           'blob_idx':blob_idx,
                           'time_frame_idx':idx}
    return cam_dict

def save_frames(frames, blob_idx, dataset='training'):
    """Save frames into pickle format. To preprocess later"""
    for frame_idx, frame in enumerate(frames):
        for camera, camera_dict in frame.items():
            with open(f'{data_destination}{dataset}/{camera}/blob_{blob_idx}_frame_{frame_idx}.pickle', 'wb') as f:
                # Pickle the 'data' dictionary using the highest protocol available.
                pickle.dump(camera_dict, f, pickle.HIGHEST_PROTOCOL)
    return None

def load_frame(frame_idx, blob_idx, dataset='training'):
    with open(f'{data_destination}{dataset}/blob_{blob_idx}.pickle', 'rb') as f:
        # Load the 'data' dictionary using the highest protocol available.
        return pickle.load(f, pickle.HIGHEST_PROTOCOL)


# Retrieve frames from selected files to download
def get_and_strip_frames_from_one_blob(downloaded_blob, blob_idx):
    # Load into tf record dataset
    dataset = tf.data.TFRecordDataset(downloaded_blob, compression_type='')
    frames = []
    for idx, data in enumerate(dataset):
        frame = open_dataset.Frame()
        frame.ParseFromString(bytearray(data.numpy()))
        # Function to strip away LIDAR and other garbage from frame
        frame = strip_frame(frame, idx, blob_idx)
        frames.append(frame)
    return frames

def download_process_save_1_blob(blob, blob_idx, dataset='training'):
    """Like dem descriptive func names eh?"""

    print(f"Downloading blob_{blob_idx}")
    blob_fname = download_blob(blob, blob_idx)
    
#     print(f"Getting and stripping all frames from blob_{blob_idx}")
    frames = get_and_strip_frames_from_one_blob(blob_fname, blob_idx)

#     print(f"Saving frames for blob {blob_idx}")
    save_frames(frames, blob_idx, dataset)

#     print(f'No longer need tfrecord blob_{blob_idx}. Deleting now.')
    os.remove(f'data/blob_{blob_idx}.tfrecord')

    return f"blob_{blob_idx}"
          

In [28]:
# Initialise a client
storage_client = storage.Client(project= project) #storage.Client(project= "Waymo3DObjectDetection", credentials=credentials)
# Create a bucket object for our bucket
bucket = storage_client.get_bucket(bucket_name)
# Get blob files in bucket
blobs = [blob for blob in storage_client.list_blobs(bucket_name, prefix='validation/')]

n_blobs = len(blobs) # Number of blobs in the training dataset
print(f'Total number of blobs is {n_blobs}')


/usr/local/lib/python3.6/dist-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Total number of blobs is 202


In [29]:
# TRAINING
start = time.time()
downloaded_blobs = []

# Start from idx 610
thread_iterable = [(blob,blob_idx, 'validation') for blob_idx, blob in enumerate(blobs)]

# thread_iterable = ((blob,blob_idx, 'training') for blob_idx, blob in enumerate(blobs))

with concurr.ThreadPoolExecutor(max_workers = 2) as executor:

    results = executor.map(lambda args: download_process_save_1_blob(*args), thread_iterable)
    for r in results:
        print(f'\n Time elapsed {time.time() - start}')
        downloaded_blobs.append(r)
        
end = time.time()
print(f'Total time taken {end - start}')


 Time elapsed 19.75239658355713

 Time elapsed 21.364221811294556

 Time elapsed 41.881749868392944


 Time elapsed 41.884921073913574
 Time elapsed 58.90697193145752


 Time elapsed 61.08712816238403
 Time elapsed 76.28627276420593


 Time elapsed 85.01444435119629
 Time elapsed 97.5887930393219


 Time elapsed 101.1943039894104
 Time elapsed 115.45931386947632

 Time elapsed 128.10792994499207

 Time elapsed 130.71124076843262


 Time elapsed 145.56687951087952

 Time elapsed 145.56696963310242
 Time elapsed 163.14707493782043

 Time elapsed 171.65074944496155

 Time elapsed 176.26040601730347

 Time elapsed 190.33926391601562

 Time elapsed 194.36711478233337

 Time elapsed 210.7631974220276


 Time elapsed 213.79596734046936
 Time elapsed 227.51355934143066

 Time elapsed 237.44053626060486

 Time elapsed 243.24248504638672

 Time elapsed 255.82473945617676

 Time elapsed 265.1197261810303

 Time elapsed 276.4301435947418

 Time elapsed 284.9307794570923

 Time elapsed 296.55665206

 Time elapsed 1461.321121931076

 Time elapsed 1469.8529164791107


 Time elapsed 1487.2735691070557

 Time elapsed 1487.2736513614655
 Time elapsed 1502.7554144859314

 Time elapsed 1505.1350963115692

 Time elapsed 1525.766099691391


 Time elapsed 1525.7668397426605
 Time elapsed 1540.658563375473


 Time elapsed 1551.5301611423492
 Time elapsed 1559.035404920578

 Time elapsed 1568.6766300201416

 Time elapsed 1578.9145395755768

 Time elapsed 1590.2959082126617

 Time elapsed 1600.2218596935272

 Time elapsed 1607.4462540149689


 Time elapsed 1624.5390865802765

 Time elapsed 1624.5392315387726
 Time elapsed 1644.954743385315

 Time elapsed 1644.9549226760864


 Time elapsed 1661.3196687698364

 Time elapsed 1663.2481470108032
 Time elapsed 1684.2820358276367


 Time elapsed 1685.183453798294
 Time elapsed 1709.0783922672272

 Time elapsed 1709.0790600776672

 Time elapsed 1716.7150712013245

 Time elapsed 1727.494222164154

 Time elapsed 1740.1081702709198

 Time elapsed 1743.38

In [ ]:
# VALIDATION
# val blobs
blobs = [blob for blob in storage_client.list_blobs(bucket_name, prefix='validation/')]

start = time.time()
downloaded_blobs = []

thread_iterable = ((blob,blob_idx, 'validation') for blob_idx, blob in enumerate(blobs))

with concurr.ThreadPoolExecutor(max_workers = 2) as executor:

    results = executor.map(lambda args: download_process_save_1_blob(*args), thread_iterable)
    for r in results:
        print(f'\n Time elapsed {time.time() - start}')
        downloaded_blobs.append(r)
        
end = time.time()
print(f'Total time taken {end - start}')


In [12]:
import os
x = sorted(os.listdir('data/training/SIDE_LEFT'))
set(i[:9] for i in x)

{'blob_0_fr',
 'blob_100_',
 'blob_101_',
 'blob_102_',
 'blob_103_',
 'blob_104_',
 'blob_105_',
 'blob_106_',
 'blob_107_',
 'blob_108_',
 'blob_109_',
 'blob_10_f',
 'blob_110_',
 'blob_111_',
 'blob_112_',
 'blob_113_',
 'blob_114_',
 'blob_115_',
 'blob_116_',
 'blob_117_',
 'blob_118_',
 'blob_119_',
 'blob_11_f',
 'blob_120_',
 'blob_121_',
 'blob_122_',
 'blob_123_',
 'blob_124_',
 'blob_125_',
 'blob_126_',
 'blob_127_',
 'blob_128_',
 'blob_129_',
 'blob_12_f',
 'blob_130_',
 'blob_131_',
 'blob_132_',
 'blob_133_',
 'blob_134_',
 'blob_135_',
 'blob_136_',
 'blob_137_',
 'blob_138_',
 'blob_139_',
 'blob_13_f',
 'blob_140_',
 'blob_141_',
 'blob_142_',
 'blob_143_',
 'blob_144_',
 'blob_145_',
 'blob_146_',
 'blob_147_',
 'blob_148_',
 'blob_149_',
 'blob_14_f',
 'blob_150_',
 'blob_151_',
 'blob_152_',
 'blob_153_',
 'blob_154_',
 'blob_155_',
 'blob_156_',
 'blob_157_',
 'blob_158_',
 'blob_159_',
 'blob_15_f',
 'blob_160_',
 'blob_161_',
 'blob_162_',
 'blob_163_',
 'blob